In [1]:
import pandas as pd
data = pd.read_csv('train_tweets.csv')
data.sample(5)

C:\Users\kote\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kote\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\kote\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


,id,label,tweet
4095,4096,0,weâre to reveal something for us proud sou...
16444,16445,0,what? that is stereotyping. and you know it is...
6540,6541,0,@user thanks. i will. just called mohammed on ...
21018,21019,0,@user that's both and #hysterical! i'm kind ...
19880,19881,0,i'm moving to toronto in a few monthsð


1. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим
функцию:
● для того, чтобы найти все вхождения паттерна в тексте, необходимо
использовать re.findall(pattern, input_txt)
● для для замены @user на пробел, необходимо использовать re.sub()

2. Изменим регистр твитов на нижний с помощью .lower().


In [2]:
import re

In [76]:
def lower_dogs(input_txt):
    pattern = "@[\w]*"
    item = re.findall(pattern, input_txt)
    
    try:
        return re.sub(item[0],"", input_txt.lower())
    except:
        return input_txt.lower()
    
   

In [77]:
data["tweet"][:5].apply(lower_dogs)

0      when a father is dysfunctional and is so sel...
1      thanks for #lyft credit i can't use cause th...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

3. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя
apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в
тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в
качестве ключа (сокращенного слова), то заменить ключ на значение (полную
версию слова).


In [88]:
import spacy
texts = ["what's", "must've", "told"]
nlp = spacy.load("en_core_web_lg")

for text in texts:
        doc = nlp(text)
        lemmatized_text = ' '.join([token.lemma_ for token in doc])
        print(lemmatized_text)

what be
must 've
tell


In [101]:
def apostrophe(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    #emoji
    phrase = re.sub(r"\:\)", " happy", phrase)
    phrase = re.sub(r"\:\(", " sad", phrase)
    
    #Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.
    phrase = re.sub(r'[^\w\s]', " ", phrase)
    
    #Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
    phrase = re.sub( r'[^a-zA-Z0-9]', " ", phrase)
    
    #Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.
    phrase = re.sub( r'[^a-zA-Z]', " ", phrase)
    
    #Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).
    phrase = ' '.join([w for w in phrase.split() if len(w)>1]) 
    return phrase

test = ":) :(" #data["tweet"][1]
print(test)
print(apostrophe(test))

:) :(
happy sad


In [107]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [108]:
from nltk.tokenize import word_tokenize

In [109]:
data["token"] = data["tweet"].apply(word_tokenize)

In [112]:
data["tweet"][2]

'  bihday your majesty'

Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.
Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.
Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.
Сохраним результат предобработки в pickle-файл.


In [115]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

In [127]:
def del_stop_words(words):
   
    return [word for word in words if not word in stop_words]
    

In [128]:
data["token_filtered"] = data["token"].apply(del_stop_words)

In [137]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [138]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [140]:
def stemmer_words(words):
    return [stemmer.stem(word) for word in words]

def lemmatizer_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

In [136]:
data["stemmer_token_filtered"] = data["token_filtered"].apply(stemmer_words)

In [141]:
data["lemmatizer_token_filtered"] = data["token_filtered"].apply(lemmatizer_words)

In [142]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)